# EtaPi PWA
based on demo_JPAC_fit.ipynb

In [ ]:
import math
total=0
n=9
for k in range(1,n+1):
    total+=math.comb(n,k)
print(total)

In [ ]:
import sys
sys.version

In [ ]:
import PyPWA as pwa
import pickle
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool
import pandas
import numpy as npy
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import emcee
import corner
import os
import random
import time
import collections

from sklearn.neighbors import KernelDensity

In [ ]:
import matplotlib
SMALL_SIZE = 14
MEDIUM_SIZE = 20
BIGGER_SIZE = 26

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Create results directory

In [ ]:
# try to stick to [epsilon][L][M]_[epsilon][L][M]_...
outputpath = "example"
try:
    os.makedirs(outputpath)
except OSError:
    if os.path.isdir(outputpath):
        print(f"Directory {outputpath} already exists.")
    else:
        print (f"Creation of the directory {outputpath} failed.")
else:
    print (f"Successfully created the directory {outputpath}.")

# Read processed data
pre-processed by process_data.py

In [ ]:
# data_full = pwa.read("./zPhase1_data/phase1_data_data.csv")
# mc_full = pwa.read("./zPhase1_data/phase1_flat_data.csv")

# data_full_weight = pwa.read("./zPhase1_data/phase1_data_weights.csv")
# mc_full_weight = pwa.read("./zPhase1_data/phase1_flat_weights.csv")

baseFolder="/d/grid17/ln16/myDSelector/amptools/pypwa/mcmc_pypwa/zMalte_kmatrix_hel/"
data_full = pwa.read(baseFolder+"malte_kmatrix_data.csv")
mc_full = pwa.read(baseFolder+"flat_2018_8_data_000.csv")

data_full_weight = pwa.read(baseFolder+"malte_kmatrix_weights.csv")
mc_full_weight = pwa.read(baseFolder+"flat_2018_8_weights_000.csv")

In [ ]:
data_weight=data_full_weight#[data_full["beamAngle"]==0]
data=data_full#[data_full["beamAngle"]==0]

mc_weight=mc_full_weight#[mc_full["beamAngle"]==0]
mc=mc_full#[mc_full["beamAngle"]==0]

In [ ]:
pd_data=pandas.DataFrame(mc)
fig,ax=plt.subplots(1,1,figsize=(16,16))
pd_data.hist(ax=ax,bins=100,weights=mc_weight["weightASBS"])

pd_data=pandas.DataFrame(data)
fig,ax=plt.subplots(1,1,figsize=(16,16))
pd_data.hist(ax=ax,bins=100,weights=data_weight["weightASBS"])

In [ ]:
_=plt.hist2d(pd_data.mass,npy.cos(pd_data.theta),bins=(75,75),weights=data_weight["weightASBS"])

# Define Waves for Fit (and input initial values of minuit and fitted parameters).

### Fit real and imaginary parts

In [ ]:
def setupWave(waveset):
    Vs = {"errordef": 1}
    initial = []
    for waveConfig in waveset:
        wave, anchor = waveConfig
        for param_type in ["r", "i"]:
            initial.append(f"{param_type}.{wave}")
            if anchor and param_type=="i":
                Vs[f"{param_type}.{wave}"] = 0
                Vs[f"fix_{param_type}.{wave}"]=True
            else:
                Vs[f"{param_type}.{wave}"] = random.uniform(-200,200)
    #             Vs[f"limit_{param_type}.{wave}"] = [-500, 1500.]
    #             Vs[f"error_{param_type}.{wave}"] = .1
    return initial, Vs

In [ ]:
waveset=[
    # S-waves
    ["1.0.0",True],
    ["-1.0.0",True],
    
    # D-waves
    ["1.2.0",False],
    ["1.2.1",False],
    ["1.2.2",False],
    ["-1.2.-1",False],
    ["-1.2.0",False],
    ["-1.2.1",False],
    
    # P-waves
    ["1.1.0",False],
    ["1.1.1",False],
    ["-1.1.0",False],
    ["-1.1.1",False]
]

initial,Vs = setupWave(waveset)
    
with open(f'{outputpath}/wave_set.csv', 'w') as file:
    for key in Vs.keys():
        file.write(f"{key},{Vs[key]}\n")

### Define amplitude and phase

In [ ]:
# Vs = {"errordef": 1}

# initial = []
# for param_type in ["a", "p"]: #try to fit magnitude and angle
#     initial.append(f"{param_type}.1.0.0")
#     initial.append(f"{param_type}.1.1.0")
#     initial.append(f"{param_type}.1.1.1")
#     initial.append(f"{param_type}.1.2.0")
#     initial.append(f"{param_type}.1.2.1")
#     initial.append(f"{param_type}.1.2.2")
# #     initial.append(f"{param_type}.1.2.-1")
# #     initial.append(f"{param_type}.1.2.-2")
    
# Vs[f"a.1.0.0"] = 10
# Vs[f"limit_a.1.0.0"] = [0, 1500.]
# Vs[f"error_a.1.0.0"] = .1

# Vs[f"a.1.1.0"] = 10
# Vs[f"limit_a.1.1.0"] = [0, 1500.]
# Vs[f"error_a.1.1.0"] = .1

# Vs[f"a.1.1.1"] = 10
# Vs[f"limit_a.1.1.1"] = [0, 1500.]
# Vs[f"error_a.1.1.1"] = .1

# Vs[f"a.1.2.0"] = 10
# Vs[f"limit_a.1.2.0"] = [0, 1500.]
# Vs[f"error_a.1.2.0"] = .1

# Vs[f"a.1.2.1"] = 10
# Vs[f"limit_a.1.2.1"] = [0, 1500.]
# Vs[f"error_a.1.2.1"] = .1

# Vs[f"a.1.2.2"] = 10
# Vs[f"limit_a.1.2.2"] = [0, 1500.]
# Vs[f"error_a.1.2.2"] = .1 

# # Vs[f"a.1.2.-1"] = 10
# # Vs[f"limit_a.1.2.-1"] = [0, 1500.]
# # Vs[f"error_a.1.2.-1"] = .1

# # Vs[f"a.1.2.-2"] = 10
# # Vs[f"limit_a.1.2.-2"] = [0, 1500.]
# # Vs[f"error_a.1.2.-2"] = .1

# Vs[f"p.1.0.0"] = 0
# Vs[f"limit_p.1.0.0"] = [0, 2*npy.pi]
# Vs[f"error_p.1.0.0"] = .01
# Vs[f"fix_p.1.0.0"] = True

# Vs[f"p.1.1.0"] = 0
# Vs[f"limit_p.1.1.0"] = [0, 2*npy.pi]
# Vs[f"error_p.1.1.0"] = .01

# Vs[f"p.1.1.1"] = 0
# Vs[f"limit_p.1.1.1"] = [0, 2*npy.pi]
# Vs[f"error_p.1.1.1"] = .01

# Vs[f"p.1.2.0"] = 0
# Vs[f"limit_p.1.2.0"] = [0, 2*npy.pi]
# Vs[f"error_p.1.2.0"] = .01

# Vs[f"p.1.2.1"] = 0
# Vs[f"limit_p.1.2.1"] = [0, 2*npy.pi]
# Vs[f"error_p.1.2.1"] = .01

# Vs[f"p.1.2.2"] = 0
# Vs[f"limit_p.1.2.2"] = [0, 2*npy.pi]
# Vs[f"error_p.1.2.2"] = .01 

# # Vs[f"p.1.2.-1"] = 0
# # Vs[f"limit_p.1.2.-1"] = [0, 2*npy.pi]
# # Vs[f"error_p.1.2.-1"] = .01

# # Vs[f"p.1.2.-2"] = 0
# # Vs[f"limit_p.1.2.-2"] = [0, 2*npy.pi]
# # Vs[f"error_p.1.2.-2"] = .01
    

# print(Vs)

# with open(f'{outputpath}/wave_set.csv', 'w') as file:
#     for key in Vs.keys():
#         file.write(f"{key},{Vs[key]}\n")

In [ ]:
import AmplitudeJPACfit
amplitude = AmplitudeJPACfit.FitAmplitude(initial)
# import AmplitudeJPACfitAngles
# amplitude = AmplitudeJPACfitAngles.FitAmplitude(initial)

# Binning of the data/monte-carlo and __define amplitude (function) to fit__
> Here the user defines number of bins, variable to be binned and range

In [ ]:
#Define number of bins 
nbins = 30 #45
mmin=0.8 #0.7
mmax=2.0 #2.5
binsda = pwa.bin_by_range(data, "mass", nbins, mmin, mmax)
binsmc = pwa.bin_by_range(mc, "mass", nbins, mmin, mmax)
binsda_w = pwa.bin_by_range(data_weight, "mass", nbins, mmin, mmax)
binsmc_w = pwa.bin_by_range(mc_weight, "mass", nbins, mmin, mmax)

In [ ]:
for i in range(len(binsda)):
    print("bin{0} (data, mc): ({1},{2})".format(i,len(binsda[i]),len(binsmc[i])))

Plot events per bin for visualisation and to make sure there are enough events per bin

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(10,5))

### DATA
counts=[]
centers=[]
for ibin,bin in enumerate(binsda):
    counts.append(npy.sum(binsda_w[ibin]["weightASBS"]))
    centers.append(npy.mean(bin["mass"]))

# Add yerr to argment list when we have errors
ax[0].step(centers,counts,label='data')
ax[0].legend(loc='upper right')

### FLAT
counts=[]
centers=[]
for ibin,bin in enumerate(binsmc):
    counts.append(npy.sum(binsmc_w[ibin]["weightASBS"]))
    centers.append(npy.mean(bin["mass"]))

# Add yerr to argment list when we have errors
ax[1].step(centers,counts,label='data')
ax[1].set_ylim(bottom=0)
ax[1].legend(loc='upper right')

In [ ]:
from IPython.display import display
intensities = []
for the_bin in binsda:
    amplitude.setup(the_bin)

# Fitting with Minuit and Extended LogLikelihood
> Look at other possibilities through pypwa (use the ?pwa command\
> or see https://pypwa.jlab.org or https://github.com/JeffersonLab/PyPWA)

In [ ]:
def spawnFit(index):
    #############
    # Arguments to PyPWA's minuit
    # https://github.com/JeffersonLab/PyPWA/blob/342026d5ea9ac37c437081c495142269cbcbe112/PyPWA/libs/fit/minuit.py
    #############
    # parameters : List[str]
    #     The names of the parameters for iminuit to use
    # settings : Dict[str, Any]
    #     The settings to be passed to iminuit. Look into the documentation
    #     for iminuit for specifics
    # likelihood : Likelihood object from likelihoods or single function
    # set_up : float
    #     Set to 1 for log-likelihoods, or .5 for Chi-Squared
    # strategy : int
    #     Fitting strategy. Defaults to 1. 0 is slowest, 2 is fastest/
    # num_of_calls : int
    #     A suggested max number of calls to minuit. This may or may not
    #     be respected.

    # MINUIT itself contains arguments. i.e.
    # to get the functions minimum we can use fval on the optimizer:
    # https://iminuit.readthedocs.io/en/stable/reference.html
    
    ##############
    # Likelihood arguments
    # https://github.com/JeffersonLab/PyPWA/blob/342026d5ea9ac37c437081c495142269cbcbe112/PyPWA/libs/fit/likelihoods.py
    ##############
    
    start=time.time() 
    results[index]=[]
    for ifit in range(nfits):
        initial,Vs = setupWave(waveset)
        with pwa.LogLikelihood(
            amplitude, binsda[index], binsmc[index], quality_factor=binsda_w[index]["weightASBS"],
            quality_factor_mc=binsmc_w[index]["weightASBS"],
            #generated_length=len(binsmc_w[index]["weightASBS"]),
            num_of_processes=1) as Likelihood:
            results[index].append(pwa.minuit(initial, Vs, Likelihood, 1, 1, 5000))
    end=time.time()
    print(f"Fit #{index} done in {end-start}s")

print("Starting pool of workers...")
globalStart = time.time()
results = {}
nfits=20 # fits per bin
nthreads=25
assert nthreads <= nbins

with ThreadPool(nthreads) as p:
    p.map(spawnFit,range(len(binsda)))
globalEnd = time.time()

# sort the dictionary of minimizers by the keys (bin number)
results=list(collections.OrderedDict(sorted(results.items())).values())

print("Elapsed time: {}".format(globalEnd-globalStart))

In [ ]:
resultsDict={}
countbins=0
for ibin in range(len(results)):
    countbins+=1
    resultsDict[f"bin{ibin}"]={}
    for ifit,result in enumerate(results[ibin]):
        resultsDict[f"bin{ibin}"][f"fit{ifit}"]=[
            [result.fmin.is_valid,result.fmin.fval],
            results[ibin][ifit].np_values(),
            results[ibin][ifit].np_errors()
        ]
print(f"Total bins: {countbins}")

In [ ]:
print(f"number of bins: {len(resultsDict.keys())}")
for bin_key in resultsDict.keys():
    print(f"number of fits in {bin_key}: {len(resultsDict[bin_key])}")

In [ ]:
fval_idxs=[]
all_invalid_fits_idxs=[]
for ibin,bin_key in enumerate(resultsDict.keys()):
    print(bin_key)
    fval_min=npy.inf
    fval_idx=0
    n_valid=0
    for ifit,fit_key in enumerate(resultsDict[bin_key].keys()):
        fval=resultsDict[bin_key][fit_key][0][1]
        status=resultsDict[bin_key][fit_key][0][0]
        status="valid" if status else "invalid"
        print(f"  ({ifit})({status})fval: {fval}")
        if fval < fval_min:
            fval_min=fval
            fval_idx=ifit
        if status=="valid":
            n_valid+=1
    if n_valid==0:
        all_invalid_fits_idxs.append(ibin)
    print(f"  --- Best fval={fval_min} at idx={fval_idx}")
    print(f"  --- # valid status={n_valid}")
    fval_idxs.append(fval_idx)

In [ ]:
resultsBest=[]
resultsBestErrs=[]
resultsBest_optimizer=[]
for binNum,bin_key in enumerate(resultsDict.keys()):
    fval_idx=fval_idxs[binNum]
    resultsBest.append(resultsDict[bin_key][f"fit{fval_idx}"][1])
    resultsBestErrs.append(resultsDict[bin_key][f"fit{fval_idx}"][2])
    resultsBest_optimizer.append(results[binNum][fval_idx])
resultsBest=npy.array(resultsBest)
resultsBestErrs=npy.array(resultsBestErrs)

In [ ]:
failed = 0
for index, result in enumerate(resultsBest_optimizer):
    display(f"Bin #{index}:")
    display(result.get_fmin())
    try:
        display(result.get_param_states())
    except:
        failed += 1
        pass

display(f"{(failed / len(results)) * 100}% ({failed})Failed. ")

In [ ]:
# Having trouble pickling the fit results. Will just save the important values and reconstruct the necessary outputs...
resultsData={ "best":resultsBest, "best_err": resultsBestErrs, "data":resultsDict , "parameters":results[0][0].parameters }

## Writing minuit fit results

In [ ]:
pickle.dump(resultsData,open("minuit_results.pkl","wb"))

In [ ]:
final_values = pandas.DataFrame(resultsData["best"], columns=resultsData["parameters"])
final_values.to_csv(f"{outputpath}/minuit.csv", index=False)

final_errs = pandas.DataFrame(resultsData["best_err"], columns=resultsData["parameters"])
final_errs.to_csv(f"{outputpath}/minuit_err.csv", index=False)

## Reading minuit fit results

In [ ]:
resultsData=pickle.load(open("minuit_results.pkl","rb"))

# Plotting results of the fitting

*Checking the waves used (and filling waves variable for later use)*

In [ ]:
waves = amplitude.make_elm(resultsData["parameters"])

*Filling strings with wave names (for plotting)*

In [ ]:
wave = npy.empty(len(waves),dtype="int")
string = npy.empty(len(waves),dtype="U5")
for index, w in waves.iterrows():
    #print(w["l"])
    string[index]= "{}{}{}".format(w["e"],w["l"],w["m"])
    wave[index]=w.name

*Getting the bin mass values and number of events in datasample for those bins*

In [ ]:
bmass=[]
mcounts=[]
for index, bin in enumerate(binsda):
    if len(bin)==0:
        bmass.append(0.)
        mcounts.append(0.)
    else:
        bmass.append(npy.average(bin["mass"]))
        mcounts.append(npy.sum(binsda_w[index]["weightASBS"]))

# Calculating the expected number of events in a mass bin

In [ ]:
total_nExp = npy.empty(len(binsmc))
for index, the_bin, result in zip(range(len(binsmc)), binsmc, resultsData["best"]):
    amp=amplitude
    amp.setup(the_bin)
    resultMap={resultsData["parameters"][i]:result[i] for i in range(resultsData["best"].shape[1])}
    total_nExp[index] = npy.average(binsmc_w[index]["weightASBS"]*amp.calculate(resultMap))

*Plot expected number of events vs mass and data vs mass (both should agree if fitting worked)*

In [ ]:
import matplotlib.pyplot as plt
mni = npy.empty(len(total_nExp), dtype=[("mass", float), ("int", float)])
mda = npy.empty(len(mcounts), dtype=[("mass", float), ("intd", float)])
mni["mass"] = bmass
mni["int"] = total_nExp
mni = pandas.DataFrame(mni)
counts, bin_edges = npy.histogram(mni["mass"], nbins, weights=mni["int"])
mda["mass"] = bmass
mda["intd"] = mcounts
mda = pandas.DataFrame(mda)
dcounts, bin_edges = npy.histogram(mda["mass"], nbins, weights=mda["intd"])
#dcounts, bin_edges = npy.histogram(bmass, nbins, weights=total_nExp)
centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Add yerr to argment list when we have errors
yerr = npy.empty(nbins)
yerr = npy.sqrt(counts)
myerr = npy.empty(nbins)
myerr = npy.sqrt(dcounts)

#plt.errorbar(centers,counts, yerr, fmt="s",linestyle="dashed",markersize='10',label="nExp")
width=(centers[1]-centers[0])/2
plt.step(centers+width,mcounts,label="Data",linestyle='-',c='gray')
plt.scatter(centers,total_nExp,label="Fitted",linestyle='-',s=32,color='steelblue',alpha=1)
plt.legend(loc='upper right',prop={"size":20})
plt.xlabel(r"$M(\eta\pi)$",size=20)
plt.ylabel("Intensity",size=20)
plt.show()

Calculate initial intensities (in case we need to check them)

In [ ]:
intensities = []
for the_bin in binsda:
    amp.setup(the_bin)
    intensities.append(amp.calculate(Vs))
intesities=pandas.DataFrame(intensities)

# Calculate the expected number of events for each wave (vs mass)

In [ ]:
####################
# Calculate the wave content
####################  

total_nExp = npy.empty(len(binsmc))
for index, the_bin, result in zip(range(len(binsmc)), binsmc, resultsData["best"]):
    amp=amplitude
    amp.setup(the_bin)
    resultMap={resultsData["parameters"][i]:result[i] for i in range(resultsData["best"].shape[1])}
    total_nExp[index] = npy.average(binsmc_w[index]["weightASBS"]*amp.calculate(resultMap))


wave_nExp = npy.empty([len(waves),len(binsmc)],npy.longdouble)
for index, the_bin, result in zip(range(len(binsmc)), binsmc, resultsData["best"]):
    amp.setup(the_bin)
    resultMap={resultsData["parameters"][i]:result[i] for i in range(resultsData["best"].shape[1])}
    for i in range(len(waves)):
        wave_nExp[i][index] = npy.average(binsmc_w[index]["weightASBS"]*amp.calculate_wave(resultMap,i))

# Expected number of events vs mass for each wave in a same plot

In [ ]:
# look for the unique LM waves. We will plot each LM wave on a different axis
#   with different colors for the + and - reflectivity waves
lms=[]
for i,w in waves.iterrows():
    lm=f"{w['l']}_{w['m']}"
    lms.append(lm)
lms=npy.unique(npy.array(lms))
lms=[lm.split("_") for lm in lms]
lms0=[lm for lm in lms if lm[0]=='0']
lms1=[lm for lm in lms if lm[0]=='1']
lms2=[lm for lm in lms if lm[0]=='2']
lms=lms0+lms2+lms1
lms=npy.array(lms)

# Need to track which array element holds which wave
mapWaves={wave:i for i,wave in enumerate(string)}

In [ ]:
######################
# Make the final plot
######################

fig,ax=plt.subplots(3,3,figsize=(14,14),sharey=True)
ax=ax.flatten()

for iax,lm in enumerate(lms):
    ax[iax].step(centers+width,mcounts,label="Data",linestyle='-',c='gray')
    for ref in ["1","-1"]:
        if ref+lm[0]+lm[1] in mapWaves:
            i = mapWaves[ref+lm[0]+lm[1]]
            color = "indianred" if ref=="1" else "steelblue"
            ax[iax].scatter(centers,wave_nExp[i],label=string[i],c=color,s=12)
    legend=ax[iax].legend(loc='upper right', title="ε L M",prop={"size":12})
    legend.get_title().set_fontsize('20')
    ax[iax].set_xlabel(r"$M(\pi^0\eta) GeV$",size=20)
    ylims=ax[iax].set_ylim(bottom=0)
    
for iax in [0,3,6]:
    ax[iax].set_ylabel("Intensity / 40 MeV",size=20)

plt.tight_layout()

# Read fitted amplitudes from disk

In [ ]:
minuitresults = pandas.read_csv(f"{outputpath}/minuit.csv")

## Setup fit config for amp / phase
import AmplitudeJPACfitAngles


def convertToAP(samples):
    '''
    Convert samples from real/imaginary to amplitude/phase
    '''
    apsamples=npy.zeros(samples.shape)

    waves=[amp[2:] for amp in initial if amp[0]=="r"]

    for iw,wave in enumerate(waves):
        apsamples[...,2*iw]=npy.sqrt(samples[...,2*iw]**2+samples[...,2*iw+1]**2)
        apsamples[...,2*iw+1]=npy.arctan2(samples[...,2*iw+1],samples[...,2*iw])
        apsamples[...,2*iw+1][npy.abs(apsamples[...,2*iw+1]-3.14159) < 0.00001] = 0.0
    return apsamples

######################
# Define waveset
######################
apinitial = []
for val in initial:
    if val.startswith('r'):
        apinitial.append(val.replace('r.','a.'))
    else:
        apinitial.append(val.replace('i.','p.'))
    
######################
# Convert from re/im to amp/phase
######################
amplitudeAngles = AmplitudeJPACfitAngles.FitAmplitude(apinitial)
waves=[amp[2:] for amp in initial if amp[0]=="a" or amp[0]=="i"]

apminuitresults = minuitresults.copy()
for i in range(len(apminuitresults)):
    for iw,wave in enumerate(waves):
        # amp = sqrt(Re^2+Im^2)
        apminuitresults.loc[i][2*iw] = npy.sqrt(minuitresults.loc[i][2*iw]**2+minuitresults.loc[i][2*iw+1]**2)
        # phase = atan2(Im,Re)
        #apminuitresults.loc[i][2*iw+1] = npy.arctan2(minuitresults.loc[i][2*iw+1],minuitresults.loc[i][2*iw])
        apminuitresults.loc[i][2*iw+1] = npy.angle(minuitresults.loc[i][2*iw+1]+1j*minuitresults.loc[i][2*iw])

apminuitresults.columns = apinitial

In [ ]:
index=0
_label="ri" #ap
_amplitude=amplitude #amplitudeAngles
_results=minuitresults # apminuitresults
_initial=initial #apinitial
startvals = _results.loc[index].copy()
if _label == "ri":
    for p in range(len(startvals)):
        if startvals[p]!=0:
            startvals[p] = startvals[p]*(1+npy.random.rand())

In [ ]:
################################
# Configuration
################################
numwalker=100
numsteps=50*200 # the autocorrelation time seems to be around 150 for our chains
ncpu=4
nthreads=24

_label="ri" #ap
_amplitude=amplitude #amplitudeAngles
_results=minuitresults # apminuitresults
_initial=initial #apinitial
################################
################################

# Run MCMC

In [ ]:
# THINNING: (thin)ning is rarely recommended these days. There are correlations betweeen nearby samples inherent in the sample procedure.
# We can reduce the correlation by thinning the chain, basically grabbing every other Nth sample. This does reduce the chains length and therefore
# the variance of the approximation BUT does gives us better iid samples if that were every needed
# https://stats.stackexchange.com/questions/442714/why-does-thinning-work-in-bayesian-inference


def spawnMCMC(index):
#     with pwa.LogLikelihood(
#         amplitude, dbin, binsma[index], generated_length=len(binsma[index]), num_of_processes=ncpu, is_minimizer=False) as Likelihood:
    with pwa.LogLikelihood(
        _amplitude, binsda[index], binsmc[index], quality_factor=binsda_w[index]["weightASBS"],
        quality_factor_mc=binsmc_w[index]["weightASBS"],
        num_of_processes=ncpu, is_minimizer=False) as Likelihood:
        print(f"Find random start parameters for fit #{index}...")
        startpars = npy.zeros((numwalker,len(_initial)))
        for i in range(numwalker):
            a=0
            while True:
                a+=1
                
                #add some randomization here for moves to work better
                startvals = _results.loc[index].copy()
                if _label == "ri":
                    for p in range(len(startvals)):
                        if startvals[p]!=0:
                            startvals[p] = startvals[p]*(1+npy.random.uniform(-0.1,0.1))#-0.5,0.5))
                else:
                    for iw in range(0,len(waves)):
                        startvals[2*iw] = startvals[2*iw]*(1+npy.random.uniform(-0.1,0.1))#-0.5,0.5))
                        #initialise the phases with an overall offset to go round on the circle. 
                        #We will get the remainder first and then scale
                        startvals[2*iw+1] = (startvals[2*iw+1] % (2*npy.pi))*(1+npy.random.uniform(-0.1,0.1))#-0.5,0.5))
                    startvals[1]=0 #fix phase for s wave
                    startvals[3]=0 #fix phase for s wave

                nll = Likelihood(startvals.to_dict())
                if not npy.any(npy.isnan(nll)):
                    break
                    
                    
            # print(f"After {a} steps found nll={nll}.") #debugging
            startpars[i] = list(startvals)

        cov =  1 #npy.array([0.002,0.002,0.002,0.005,0.005,0.005,0.005,0.005,0.005,1E5])
        if _label == "ri":  
            parlimits = [(-1000,1000),(0,0),(-1000,1000),(0,0)] # set the s-waves to be real
            for i in range(4,len(_initial)):
                parlimits.append((-1000,1000))
        else:
            parlimits=[[] for i in range(2*len(waves))]
            for iw in range(0,len(waves)):
                parlimits[2*iw]=(0,1500) 
                if iw<2: # fix the s-waves
                    parlimits[2*iw+1]=(0,0)
                else:
                    parlimits[2*iw+1]=(0,2*npy.pi) 

        print(f"Start chain for bin {index} ...")
        # https://emcee.readthedocs.io/en/stable/user/moves/#moves-user
        # Sample from a set of moves. Moves are broken up into two paths {RedBlueMove, MHMove} which are parallel and non-parallel I think. These are two
        #         base abstract classes that other moves inherit from. GaussianMove is a MHMove with a Gaussian proposal function
        # StretchMove - Moving the walker based on a complementary walker
        # DE = Differential evolution, Multiple chains are run in parallel. The current move depends on two other chains. 
        #           Successful sampling requires at least 2*d chains where d is the dimensionality of the posterior
        # DESnooker applies a snooker move which allows for shorter chains. The move respects past states of other chains and thus allows for parallelization
        optimizer = pwa.mcmc(
                        _initial,
                        likelihood=Likelihood,
                        nsteps=numsteps,
                        startpars=startpars,
                        parlimits=parlimits,
                        nwalker=numwalker,
                        #emceemoves=[(emcee.moves.StretchMove(), 0.5), (emcee.moves.DEMove(), 0.3), (emcee.moves.DESnookerMove(), 0.2)])
                        emceemoves=[(emcee.moves.StretchMove(), 0.9*0.1), (emcee.moves.DEMove(), 0.9*0.9), (emcee.moves.DESnookerMove(), 0.1)])
#                              emceemoves=emcee.moves.StretchMove())
#                              emceemoves=emcee.moves.GaussianMove(cov,'vector'))


    _resultsMCMC[index]=optimizer
    
    acceptanceFraction=npy.mean(optimizer.acceptance_fraction)
    print("Mean acceptance fraction: {0:.3f}".format(npy.mean(optimizer.acceptance_fraction)))
    acceptanceFractions[index]=acceptanceFraction
    try:
        autocorrelationTime=npy.mean(optimizer.get_autocorr_time()) # will raise an error if autocorrelation too big, can quite=True it
        print("Mean autocorrelation time: {0:.3f} steps".format(autocorrelationTime))
        autocorrelationTimes[index]=autocorrelationTime
    except:
        print("Couldn't get autocorrelation time.")
        autocorrelationTimes[index] = -1
    
#     break #only one bin


print("Starting pool of workers for mcmc...")
globalStart = time.time()
_resultsMCMC = {}
acceptanceFractions = {}
autocorrelationTimes = {}

with ThreadPool(nthreads) as p:
    p.map(spawnMCMC,range(len(binsda)))
globalEnd = time.time()

# sort the dictionary of minimizers by the keys (bin number)
_resultsMCMC=list(collections.OrderedDict(sorted(_resultsMCMC.items())).values())
acceptanceFractions=list(collections.OrderedDict(sorted(acceptanceFractions.items())).values())
autocorrelationTimes=list(collections.OrderedDict(sorted(autocorrelationTimes.items())).values())

print("Elapsed time: {}".format(globalEnd-globalStart))


In [ ]:
results_pkl = {}
results_pkl["chain"] = [result.get_chain(discard=0) for result in _resultsMCMC]
#results_pkl["autocorr"] = [result.get_autocorr_time() for result in _resultsMCMC]
results_pkl["acceptance_fraction"] = [result.acceptance_fraction for result in _resultsMCMC]

pickle.dump(results_pkl, open("resultsMCMC.pkl","wb"))

# for i in range(len(binsda)):
#     samples = resultsMCMC[i].get_chain(discard=0)
#     pwa.write(f"{outputpath}/mcmc_mix_{numwalker}_{numsteps}_bin{i}.npy",samples)

# Read MCMC results

In [ ]:
# nbins=45
# samplesList=[]
# for i in range(nbins):
#     samples=npy.load(f"{outputpath}/mcmc_mix_{numwalker}_{numsteps}_bin{i}.npy")
#     samplesList.append(samples)

In [ ]:
#resultsMCMC = pickle.load(open("resultsMCMC.pkl","rb"))
resultsMCMC = pickle.load(open("resultsMCMC_15ksteps_100walkers_10percRnd.pkl","rb"))

samplesList=resultsMCMC["chain"]

## MCMC diagnostic plots - autocorr

### Real and imaginary parts

In [ ]:
# These two are equivalent
# NOTE THAT integrated_time ONLY WORKS IF IN ORDER [STEPS, WALKERS, PARAMS]
# resultsMCMC[0].get_autocorr_time()
# emcee.autocorr.integrated_time(resultsMCMC[0].get_chain(discard=0))

binNum=0

minstep=100
maxstep=resultsMCMC["chain"][0].shape[0] #15000
widthstep=100
steps=range(minstep,maxstep,widthstep)

autocorrs=[]
for step in steps:
    #autocorrs.append(emcee.autocorr.integrated_time(resultsMCMC[binNum].get_chain(discard=0)[:step,:,:],quiet=True))
    autocorrs.append(emcee.autocorr.integrated_time(resultsMCMC["chain"][binNum][:step,:,:],quiet=True))
    
autocorrs=npy.stack(autocorrs,axis=1)

In [ ]:
fig,ax=plt.subplots(2,2,figsize=(16,12))

for iparam in range(autocorrs.shape[0]):
    _=ax[0,0].plot(steps,autocorrs[iparam])
ax[0,0].set_xlabel("Step")
ax[0,0].set_ylabel("Integrated Autocorrelation Time")
ax[0,0].set_title(f"Mass Bin: {binNum}",size=30)

iparam=0
for iwalker in range(5):
    _=ax[0,1].plot(resultsMCMC["chain"][binNum][:,iwalker,iparam],label="walker "+str(iwalker))
ax[0,1].set_xlabel("step")
ax[0,1].set_title(f"parameter: {initial[iparam]}",size=30)
ax[0,1].legend(prop={"size":16})

cmap = matplotlib.cm.get_cmap('Spectral')
ax[1,0].hist(results_pkl["acceptance_fraction"],bins=25,stacked=True,color=list(map(cmap,npy.linspace(0,1,45))))
ax[1,0].set_xlabel("acceptance probability")
ax[1,0].set_title("Stacked over mass bins",size=30)

plt.tight_layout()

### Amplitude and phases

In [ ]:
# These two are equivalent
# resultsMCMC[0].get_autocorr_time()
# emcee.autocorr.integrated_time(resultsMCMC[0].get_chain(discard=0))

binNum=0
minstep=100
maxstep=resultsMCMC["chain"][0].shape[0] #15000
widthstep=100
steps=range(minstep,maxstep,widthstep)

apsamples=convertToAP(resultsMCMC["chain"][binNum])

autocorrs_ap=[]
for step in steps:
    #autocorrs[iwalker].append(emcee.autocorr.integrated_time(flatSamplesList[:step,0],quiet=True)[0])
    autocorrs_ap.append(emcee.autocorr.integrated_time(apsamples[:step,:,:],quiet=True))
    
autocorrs_ap=npy.stack(autocorrs_ap,axis=1)

In [ ]:
fig,ax=plt.subplots(2,2,figsize=(16,12))

for iparam in range(autocorrs_ap.shape[0]):
    _=ax[0,0].plot(steps,autocorrs_ap[iparam])
ax[0,0].set_xlabel("Step")
ax[0,0].set_ylabel("Integrated Autocorrelation Time")
ax[0,0].set_title(f"Mass Bin: {binNum}",size=30)

iparam=0
for iwalker in range(5):
    _=ax[0,1].plot(apsamples[:,iwalker,iparam],label="walker "+str(iwalker))
ax[0,1].set_xlabel("step")
ax[0,1].set_title(f"parameter: {apinitial[iparam]}",size=30)
ax[0,1].legend(prop={"size":16})

cmap = matplotlib.cm.get_cmap('Spectral')
ax[1,0].hist(results_pkl["acceptance_fraction"],bins=25,stacked=True,color=list(map(cmap,npy.linspace(0,1,45))))
ax[1,0].set_xlabel("acceptance probability")
ax[1,0].set_title("Stacked over mass bins",size=30)

plt.tight_layout()

# Compare MAP and MLE

In [ ]:
###############################
# Gather the MAP solutions for all parameters for all bins
###############################

# Some configuration
samplesList=resultsMCMC["chain"]
nparams=resultsMCMC["chain"][0].shape[2]
burnin=5000 #10000
maxsteps=10000
verbose=False

def getMAPsolutions(binNum):
    lowMass=0.7+0.04*binNum
    uppMass=0.7+0.04*(binNum+1)
    print("(Thread {0:.0f})Mass range: {1:.2f},{2:.2f}".format(binNum,lowMass,uppMass))
    flatsample=samplesList[binNum][burnin:maxsteps,...].reshape((maxsteps-burnin)*numwalker,24)
    flatsample=convertToAP(flatsample)

    for idx in range(nparams):
        wave=flatsample[:,idx]

        X = npy.histogram(wave,bins=100)
        centers=X[1][:-1]+(X[1][1]-X[1][0])/2
        
        if wave.std() != 0: # bandwidth must be > 0. wave.std not always guaranteed. i.e. if the wave was fixed to be real
            kde = KernelDensity(kernel='gaussian', bandwidth=wave.std()/4).fit(wave.reshape(-1,1))
            density = npy.exp(kde.score_samples(centers.reshape(-1,1)))
            map_solution=centers[npy.argmax(density)]
        else:
            density=X[0]/sum(X[0])
            map_solution=0
        
        # Fill dictionaries
        map_solutions[idx][binNum]=map_solution
        densities[idx][binNum]=density
        centerses[idx][binNum]=centers
        

globalStart = time.time()

# Pass getMAPsolutions to all the threads
map_solutions=[{} for idx in range(nparams)]
densities=[{} for idx in range(nparams)]
centerses=[{} for idx in range(nparams)]
nthreads=24
with ThreadPool(nthreads) as p:
    p.map(getMAPsolutions,range(len(binsda)))
    
globalEnd = time.time()

for i in range(nparams):
    map_solutions[i]=list(collections.OrderedDict(sorted(map_solutions[i].items())).values())
    densities[i]=list(collections.OrderedDict(sorted(densities[i].items())).values())
    centerses[i]=list(collections.OrderedDict(sorted(centerses[i].items())).values())

print("Elapsed time: {}".format(globalEnd-globalStart))

In [ ]:
###########################################################
# Plotting specific waves with the following labels attached
# Useful to visualize the difference between MAP and MLE
###########################################################
idx_of_D2_amp=npy.where(npy.array(apinitial)=="a.1.2.2")[0][0]
idx_of_D2_phase=npy.where(npy.array(apinitial)=="p.1.2.2")[0][0]
labels=["D2 intensity", "D2 phase"]

nbins=45
nrows=int(npy.ceil(nbins/2))
fig,ax=plt.subplots(nrows,4,figsize=(16, 4*nrows))
ax=ax.flatten()

for binNum in range(nbins):
    flatsample=samplesList[binNum][burnin:maxsteps,...].reshape((maxsteps-burnin)*numwalker,24)
    flatsample=convertToAP(flatsample)
    
    lowMass=0.7+0.04*binNum
    uppMass=0.7+0.04*(binNum+1)

    for i,idx in enumerate([idx_of_D2_amp, idx_of_D2_phase]):
        wave=flatsample[:,idx]
    
        map_solution=[ele[binNum] for ele in map_solutions][idx]
        density=densities[idx][binNum]
        centers=centerses[idx][binNum]
        
        _=ax[2*binNum+i].hist(wave,bins=50,density=True,alpha=0.6)
        ax[2*binNum+i].plot(centers,density,linewidth=3,c="royalblue")
        ax[2*binNum+i].axvline(map_solution,c='indianred',linestyle='--',linewidth=3)
        ax[2*binNum+i].axvline(apminuitresults.iloc[binNum,idx],c='darkorange',linestyle="--",linewidth=3)
        for fit_key in resultsData["data"][f"bin{binNum}"].keys():
            alt_fitresults=resultsData["data"][f"bin{binNum}"][fit_key]
            alt_fitresults_status=alt_fitresults[0][0]
            coloralphawidth = ["black",0.8,1] if alt_fitresults_status else ["gray",0.6,0.5]
            alt_fitresults_values=convertToAP(alt_fitresults[1])[idx]
            ax[2*binNum+i].axvline(alt_fitresults_values,c=coloralphawidth[0],linestyle="--",alpha=coloralphawidth[1],linewidth=coloralphawidth[2])
        ax[2*binNum+i].set_xlabel(labels[i])
        ax[2*binNum+i].set_title(r"{0:0.2f}<Mpi0eta<{0:0.2f}".format(lowMass,uppMass),size=20)

plt.tight_layout()

In [ ]:
percentiles_16=[]
percentiles_50=[]
percentiles_84=[]
for binNum in range(nbins):
    print(f"bin{binNum}")
    flatsample=samplesList[binNum][burnin:maxsteps,...].reshape((maxsteps-burnin)*numwalker,24)
    flatsample=convertToAP(flatsample)
    percentiles_16.append([])
    percentiles_50.append([])
    percentiles_84.append([])
    for idx in range(nparams):
        wave=flatsample[:,idx]
        percentile_16, percentile_50, percentile_84 = npy.percentile(wave, [16, 50, 84])
        percentiles_16[binNum].append(percentile_16)
        percentiles_50[binNum].append(percentile_50)
        percentiles_84[binNum].append(percentile_84)
        
percentiles_16=npy.array(percentiles_16).T
percentiles_50=npy.array(percentiles_50).T
percentiles_84=npy.array(percentiles_84).T

In [ ]:
####################
# Convert map_solutions array into a list of dictionary format that is usable by amp.calculate_wave
####################
map_solution_dicts=[]
for bin_num in range(nbins):
    map_solution_dict={}
    for ipar,par in enumerate(resultsData["parameters"]):
        map_solution_dict[par] = map_solutions[ipar][bin_num]
    map_solution_dicts.append(map_solution_dict)

####################
# Make calculate the wave content
####################
waves = amplitude.make_elm(resultsData["parameters"])
wave_nExp = npy.empty([len(waves),len(binsmc)],npy.longdouble)
for index, the_bin, result in zip(range(len(binsmc)), binsmc, map_solution_dicts):
    amp.setup(the_bin)
    for i in range(len(waves)):
        wave_nExp[i][index] = npy.average(binsmc_w[index]["weightASBS"]*amp.calculate_wave(result,i))
        
######################
# Make the final plot
######################
# look for the unique LM waves. We will plot each LM wave on a different axis
#   with different colors for the + and - reflectivity waves
lms=[]
for i,w in waves.iterrows():
    lm=f"{w['l']}_{w['m']}"
    lms.append(lm)
lms=npy.unique(npy.array(lms))
lms=[lm.split("_") for lm in lms]
lms0=[lm for lm in lms if lm[0]=='0']
lms1=[lm for lm in lms if lm[0]=='1']
lms2=[lm for lm in lms if lm[0]=='2']
lms=lms0+lms2+lms1
lms=npy.array(lms)

# Need to track which array element holds which wave
mapWaves={wave:i for i,wave in enumerate(string)}

fig,ax=plt.subplots(3,3,figsize=(14,14),sharey=True)
ax=ax.flatten()

centers=[]
for ibin,bin in enumerate(binsmc):
    centers.append(npy.mean(bin["mass"]))
for iax,lm in enumerate(lms):
    ax[iax].step(centers+width,mcounts,label="Data",linestyle='-',c='gray')
    for ref in ["1","-1"]:
        if ref+lm[0]+lm[1] in mapWaves:
            i = mapWaves[ref+lm[0]+lm[1]]
            color = "indianred" if ref=="1" else "steelblue"
            ax[iax].scatter(centers,wave_nExp[i],label=string[i],c=color,s=12)
    legend=ax[iax].legend(loc='upper right', title="ε L M",prop={"size":12})
    legend.get_title().set_fontsize('20')
    ax[iax].set_xlabel(r"$M(\pi^0\eta) GeV$",size=20)
    ylims=ax[iax].set_ylim(bottom=0)
    
for iax in [0,3,6]:
    ax[iax].set_ylabel("Intensity / 40 MeV",size=20)

plt.tight_layout()

In [ ]:
####################
# Convert map_solutions array into a list of dictionary format that is usable by amp.calculate_wave
####################
percentile_16_solution_dicts=[]
percentile_50_solution_dicts=[]
percentile_84_solution_dicts=[]
for bin_num in range(nbins):
    percentile_16_solution_dict={}
    percentile_50_solution_dict={}
    percentile_84_solution_dict={}
    for ipar,par in enumerate(resultsData["parameters"]):
        percentile_16_solution_dict[par] = percentiles_16[ipar][bin_num]
        percentile_50_solution_dict[par] = percentiles_50[ipar][bin_num]
        percentile_84_solution_dict[par] = percentiles_84[ipar][bin_num]
    percentile_16_solution_dicts.append(percentile_16_solution_dict)
    percentile_50_solution_dicts.append(percentile_50_solution_dict)
    percentile_84_solution_dicts.append(percentile_84_solution_dict)


####################
# Make calculate the wave content
####################
waves = amplitude.make_elm(resultsData["parameters"])
wave_nExp_50_err = npy.empty([len(waves),len(binsmc)],list)
wave_nExp_50 = npy.empty([len(waves),len(binsmc)],npy.longdouble)
for index, the_bin, result_16, result_50, result_84 in zip(range(len(binsmc)), binsmc, percentile_16_solution_dicts, percentile_50_solution_dicts, percentile_84_solution_dicts):
    amp.setup(the_bin)
    for i in range(len(waves)):
        wave_nExp_50[i][index] = npy.average(binsmc_w[index]["weightASBS"]*amp.calculate_wave(result_50,i))
        wave_nExp_16 = npy.average(binsmc_w[index]["weightASBS"]*amp.calculate_wave(result_16,i))
        wave_nExp_84 = npy.average(binsmc_w[index]["weightASBS"]*amp.calculate_wave(result_84,i))
        wave_nExp_50_err[i][index] = [abs(wave_nExp_50[i][index]-wave_nExp_16),abs(wave_nExp_50[i][index]-wave_nExp_84)]

        
######################
# Make the final plot
######################
# look for the unique LM waves. We will plot each LM wave on a different axis
#   with different colors for the + and - reflectivity waves
lms=[]
for i,w in waves.iterrows():
    lm=f"{w['l']}_{w['m']}"
    lms.append(lm)
lms=npy.unique(npy.array(lms))
lms=[lm.split("_") for lm in lms]
lms0=[lm for lm in lms if lm[0]=='0']
lms1=[lm for lm in lms if lm[0]=='1']
lms2=[lm for lm in lms if lm[0]=='2']
lms=lms0+lms2+lms1
lms=npy.array(lms)

# Need to track which array element holds which wave
mapWaves={wave:i for i,wave in enumerate(string)}

fig,ax=plt.subplots(3,3,figsize=(14,14),sharey=True)
ax=ax.flatten()

centers=[]
for ibin,bin in enumerate(binsmc):
    centers.append(npy.mean(bin["mass"]))
for iax,lm in enumerate(lms):
    ax[iax].step(centers+width,mcounts,label="Data",linestyle='-',c='gray')
    for ref in ["1","-1"]:
        if ref+lm[0]+lm[1] in mapWaves:
            i = mapWaves[ref+lm[0]+lm[1]]
            color = "indianred" if ref=="1" else "steelblue"
            errs=npy.array([ele for ele in wave_nExp_50_err[i]]).T # need to convert to array first and then transpose it
            ax[iax].errorbar(centers,wave_nExp_50[i],yerr=errs,label=string[i],c=color,markersize=3,ls='none',fmt='o')
    legend=ax[iax].legend(loc='upper right', title="ε L M",prop={"size":12})
    legend.get_title().set_fontsize('20')
    ax[iax].set_xlabel(r"$M(\pi^0\eta) GeV$",size=20)
    ylims=ax[iax].set_ylim(bottom=0)
    
for iax in [0,3,6]:
    ax[iax].set_ylabel("Intensity / 40 MeV",size=20)

plt.tight_layout()

## Draw chains and histograms

In [ ]:
apminuitresults = minuitresults.copy()
for i in range(len(apminuitresults)):
    for iw,wave in enumerate(waves):
        amp=npy.sqrt(minuitresults.loc[i][2*iw]**2+minuitresults.loc[i][2*iw+1]**2)
        phase=npy.arctan2(minuitresults.loc[i][2*iw+1],minuitresults.loc[i][2*iw])
        # amp = sqrt(Re^2+Im^2)
        apminuitresults.loc[i][2*iw] = amp
        # phase = atan2(Im,Re)
        apminuitresults.loc[i][2*iw+1] = phase

apminuitresults.columns = apinitial

In [ ]:
def spawnPlotDrawer(index):
    print(f"Show results for bin #{index}")
    if loadFromFiles:
        samples=samplesList[index]
    else:
        samples = resultsMCMC[index].get_chain(discard=0)
    if use_apinitial:
        samples=convertToAP(samples)
        
    numsteps = len(samples)
    numpars = samples.shape[2]
    fig, axes = plt.subplots(len(_initial), figsize=(10, 70), sharex=True)
    for i in range(len(_initial)):
        ax = axes[i]
        ax.plot(samples[:,:,i])
        ax.set_ylabel(_initial[i])
        ax.set_xlim(0, numsteps)

    axes[-1].set_xlabel("step number");
    if showFigures:
        fig.show()

#     if loadFromFiles:
#         samples=samplesList[index][burnin:,...]
#     else:
#         samples = resultsMCMC[index].get_chain(discard=burnin)
#     if use_apinitial:
#         samples=convertToAP(samples)
#     fig2, axes2 = plt.subplots(len(_initial), figsize=(5, 70), sharex=False)
#     for i in range(len(_initial)):
#         ax = axes2[i]
#         ax.hist(samples[:,:,i], 100,histtype='barstacked')
#         ax.set_ylabel(_initial[i])
#     if showFigures:
#         fig2.show()

    if loadFromFiles:
        flatsample=samplesList[index][burnin:,...].reshape((numsteps-burnin)*numwalker,24)
    else:
        flatsample = resultsMCMC[index].get_chain(discard=burnin,flat=True)
    if use_apinitial:
        flatsample=convertToAP(flatsample)
    varrange = []
    for i in range(len(_initial)):
        varrange.append((flatsample[:,i].min(),flatsample[:,i].max()))
    print(f"Drawing corner plot for bin #{index}")
    fig3 = corner.corner(flatsample,
                        color='royalblue',
                        bins=50,
#                         truths=npy.mean(flatsample,axis=0),
#                         quantiles=minuitresults.loc[index],
                        range=varrange,
                        labels=_initial,
                        fill_contours=True,
                        truth_color='red',
                        label_kwargs={'fontsize':20, 'labelpad':20},
                        hist_kwargs = {'histtype':'stepfilled','alpha':1})
    if showFigures:
        fig3.show()

    # Draw mean and minuit values (from emcee documentation)
    # Extract the axes
    axes = npy.array(fig3.axes).reshape((numpars, numpars))

    # Loop over the diagonal
    for i in range(numpars):
        ax = axes[i, i]
#         ax.axvline(npy.mean(flatsample,axis=0)[i], color="k")
        ax.axvline(_results.loc[index][i], color="r")

    # Loop over the histograms
    for yi in range(numpars):
        for xi in range(yi):
            ax = axes[yi, xi]
#             ax.axvline(npy.mean(flatsample,axis=0)[xi], color="k")
            ax.axvline(_results.loc[index][xi], color="r")
#             ax.axhline(npy.mean(flatsample,axis=0)[yi], color="k")
            ax.axhline(_results.loc[index][yi], color="r")
#             ax.plot(npy.mean(flatsample,axis=0)[xi], npy.mean(flatsample,axis=0)[yi], "sk")
            ax.plot(_results.loc[index][xi], _results.loc[index][yi], "sr")
    
    #######################################################
    #################### SAVE THE FIGURES #################
    #######################################################
    print(f"Saving plots for bin #{index}")
    fig.savefig(f"{plotpath}/{saveFolder}/timeline_mix_bin{index}.pdf")
    #fig2.savefig(f"{plotpath}/{saveFolder}/hist_mix_bin{index}.pdf")
    fig3.savefig(f"{plotpath}/{saveFolder}/corner_bin{index}.pdf")

In [ ]:
def makeFolder(path):
    try:
        os.makedirs(path)
    except OSError:
        if os.path.isdir(path):
            print(f"Directory {path} already exists.")
        else:
            print (f"Creation of the directory {path} failed.")
    else:
        print (f"Successfully created the directory {path}.")

In [ ]:
# Make folders for output
plotpath=f"{outputpath}"
saveFolder="diagnostics_ap"
makeFolder(f"{plotpath}")
makeFolder(f"{plotpath}/{saveFolder}")

# Configuration for plots
burnin=500
nthreads=len(binsda)
loadFromFiles=True
showFigures=False

# Whether to show the amp/phase or re/im plots
_results=apminuitresults
_initial=apinitial
use_apinitial=True

print("Starting pool of workers to draw...")
globalStart = time.time()
with Pool(nthreads) as p:
    p.map(spawnPlotDrawer,range(len(binsda)))
globalEnd = time.time()

print("Elapsed time: {}".format(globalEnd-globalStart))

In [ ]:
##############################
##############################
# Calculate Moments
##############################
##############################

# H000,H010,H011,H020,H021,H022,H100,H110,H111,H120,H121,H122,sigma4,sigmay = amp.calculate_moments_JPAC(result.values)
#H00,H11,H10,H20,H21,H22 = amp.calculate_moments_STD()


##############################
##############################
# Calculate the phase motion
##############################
##############################

# phase = npy.empty(len(binsda))
# for index, the_bin, result in zip(range(len(binsda)), binsda, results):
#     amp.setup(the_bin)
#     phase[index] = amp.Phasediff(result.values,wave[0],wave[1])

# mnip = npy.empty(len(bmass), dtype=[("mass", float), ("intp", float)])
# mnip["mass"] = bmass
# mnip["intp"] = phase
# mnip = pandas.DataFrame(mnip)
# countsp, bin_edges = npy.histogram(mnip["mass"], nbins, weights=mnip["intp"])

# centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# # Add yerr to argment list when we have errors
# yerr = npy.empty(nbins)
# yerr = npy.sqrt(npy.abs(countsp))
# plt.errorbar(centers,countsp, yerr, fmt="o")
# #plt.xlim(.6, 2.5)